In [1]:
# !git clone https://github.com/khalooei/robustness-framework.git
# %cd robustness-framework/

In [2]:
# !pip install lightning pytorch_lightning hydra-core lightning-bolts torchattacks torchmetrics mlxtend
# !pip install git+https://github.com/fra31/auto-attack

In [1]:
from plmodel import *
import omegaconf

/home/vru/workspace/ev/khpy3/lib/python3.10/site-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:34: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  "lr_options": generate_power_seq(LEARNING_RATE_CIFAR, 11),
/home/vru/workspace/ev/khpy3/lib/python3.10/site-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:92: UnderReviewWarning: The feature FeatureMapContrastiveTask is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  contrastive_task: Union[Featur

In [2]:
cfg = omegaconf.OmegaConf.load('configs/training_cifar10.yaml')
# cfg = omegaconf.OmegaConf.load('configs/training_mnist.yaml')

cfg.global_params.devices='auto'  # you can modify it manually here
cfg.training_params.epoch=200
cfg.training_params.dataloader_workers=32
cfg.training_params.type="normal"
cfg.global_params

{'dataset': 'CIFAR10', 'architecture': 'ResNet', 'devices': 'auto'}

In [3]:
print(OmegaConf.to_yaml(cfg))

defaults:
- _self_
- override hydra/hydra_logging: disabled
- override hydra/job_logging: disabled
hydra:
  output_subdir: null
  run:
    dir: .
global_params:
  dataset: CIFAR10
  architecture: ResNet
  devices: auto
training_params:
  epoch: 200
  batch_size: 1024
  dataloader_workers: 32
  type: normal
  note: ''
  lr: 0.05
adversarial_training_params:
  eps: 0.03
  name: PGD



In [4]:
# prepare name of current execution
s_dir_name = f"{cfg.global_params.dataset}-{cfg.global_params.architecture}-{cfg.training_params.type}_e{cfg.training_params.epoch}_note-{cfg.training_params.note}_"
if cfg.training_params.type == "AT":
    s_dir_name+=f"{cfg.adversarial_training_params.name}_eps{cfg.adversarial_training_params.eps}"


# configure logging on module level, redirect to file
p = pathlib.Path(f'logs/{s_dir_name}')
p.mkdir(parents=True, exist_ok=True)
consolelogger = logging.getLogger("lightning.pytorch.core")
# consolelogger.addHandler(logging.FileHandler(f"logs/{s_dir_name}/core.log"))

s_experiment_starting_time = f"{time.strftime('%Y%m%d%H%M%S')}"

In [5]:
callbacks = [
    # save_top_k -> save the best model   >>>  best model is max in val_acc
    LearningRateMonitor(logging_interval="step"),
    ModelCheckpoint(save_top_k=2, mode="max", monitor="clean_val_acc"),  # saved best model based on Maximize the validation accuracy
    CustomTimeCallback(),
]

In [6]:
# load model
model = PLModel(cfg)

/home/vru/workspace/ev/khpy3/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/vru/workspace/ev/khpy3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [7]:
# Initiate our train/inference handler
trainer = pl.Trainer(max_epochs=cfg.training_params.epoch,
                    devices=cfg.global_params.devices,
                    num_nodes=1,
                    gpus=1,
                    # strategy= "ddp_notebook", #ddp_notebook for jupyter and "ddp" for PC/server
                    callbacks=callbacks,
                    logger=[CSVLogger(save_dir=f'logs/{s_dir_name}/',version=s_experiment_starting_time),
                            TensorBoardLogger(f'logs/{s_dir_name}/',version=s_experiment_starting_time),
                            KhalooeiLoggingLogger(save_dir=f'logs/{s_dir_name}',version=s_experiment_starting_time),],
                    accelerator='gpu', #reproducibility,
                    deterministic=True, #reproducibility,
                    inference_mode=False,
                    # auto_lr_find = True # to find better lr
                    )

/home/vru/workspace/ev/khpy3/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:478: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
/home/vru/workspace/ev/khpy3/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:498: The flag `devices=auto` will be ignored, instead the device specific number 1 will be used


In [8]:
# tuning
# from pytorch_lightning.tuner.tuning import Tuner
# tuner = Tuner(trainer)
# # lr_finder= tuner.lr_find(model, datamodule=model.train_dataloader())

In [9]:
# training
trainer.fit(model)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [2]

  | Name       | Type                | Params
---------------------------------------------------
0 | model      | _ResNet18_benchmark | 11.2 M
1 | loss       | NLLLoss             | 0     
2 | metric_acc | MulticlassAccuracy  | 0     
3 | swa_model  | AveragedModel       | 11.2 M
---------------------------------------------------
22.3 M    Trainable params
0         Non-trainable params
22.3 M    Total params
89.392    Total estimated model params size (MB)
/home/vru/workspace/ev/khpy3/lib/python3.10/site-packages/lightning_fabric/loggers/csv_logs.py:183: Experiment logs directory logs/CIFAR10-ResNet-normal_e200_note-_/lightning_logs/20231125060956 exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
2023-11-25 06:10:11,609 [INFO]: Hyperparameters:
2023-11-25 06:10:11,610 [INFO]: cfg: {'defaults': ['_self_', {'override hydra/hydra_logging': 'disabled'}, {'override hydra/job_logging': 'dis

Sanity Checking: 0it [00:00, ?it/s]

Training is starting ...


/home/vru/workspace/ev/khpy3/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:1609: The number of training batches (44) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


...

Validation: 0it [00:00, ?it/s]

2023-11-25 06:56:29,770 [INFO]: clean_val_acc: 0.8754165768623352
2023-11-25 06:56:29,772 [INFO]: clean_val_loss: 0.5923770666122437
2023-11-25 06:56:29,773 [INFO]: epoch: 198
2023-11-25 06:56:29,803 [INFO]: clean_train_acc_epoch: 0.9985555410385132
2023-11-25 06:56:29,804 [INFO]: clean_train_loss_epoch: 0.006210337392985821
2023-11-25 06:56:29,805 [INFO]: epoch: 198
2023-11-25 06:56:40,128 [INFO]: lr-SGD: 0.004002515629172365
2023-11-25 06:56:40,338 [INFO]: clean_train_acc_step: 1.0
2023-11-25 06:56:40,339 [INFO]: clean_train_loss_step: 0.003076084889471531
2023-11-25 06:56:40,341 [INFO]: epoch: 199


Validation: 0it [00:00, ?it/s]

2023-11-25 06:56:42,263 [INFO]: clean_val_acc: 0.879687488079071
2023-11-25 06:56:42,264 [INFO]: clean_val_loss: 0.5957626104354858
2023-11-25 06:56:42,264 [INFO]: epoch: 199
2023-11-25 06:56:42,299 [INFO]: clean_train_acc_epoch: 0.9989333152770996
2023-11-25 06:56:42,300 [INFO]: clean_train_loss_epoch: 0.005725202616304159
2023-11-25 06:56:42,301 [INFO]: epoch: 199


Training is finished. It took 46.47969859043757 min.


In [10]:
# results = trainer.tune(model=model, datamodule=model.train_dataloader()) # for tuning
# results['lr_find'].plot(suggest=True)
# trainer.save_checkpoint('model.ckpt') #save manually

In [11]:
# Evaluating
print('-----TEST ACC----------')
# test_acc = trainer.test(ckpt_path='best')  #dataloaders=test_loader
# best model is a model with best ones

# print('-----VAL ACC----------')
# val_acc = trainer.test(dataloaders=model.val_loader,ckpt_path='best')  #dataloaders=test_loader
# print(val_acc)
# print('-----Train ACC----------')
# train_acc = trainer.test(dataloaders=model.train_loader,ckpt_path='best')  #dataloaders=test_loader
# print(train_acc)
print('-----TEST ACC----------')
model.test_dataloader()
test_acc = trainer.test(dataloaders=model.test_loader)  #dataloaders=test_loader
print(test_acc)
# every saved log is in lightning-Logs

-----TEST ACC----------
-----TEST ACC----------


/home/vru/workspace/ev/khpy3/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:124: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [2]
2023-11-25 06:56:53,625 [INFO]: Hyperparameters:
2023-11-25 06:56:53,629 [INFO]: cfg: {'defaults': ['_self_', {'override hydra/hydra_logging': 'disabled'}, {'override hydra/job_logging': 'disabled'}], 'hydra': {'output_subdir': None, 'run': {'dir': '.'}}, 'global_params': {'dataset': 'CIFAR10', 'architecture': 'ResNet', 'devices': 'auto'}, 'training_params': {'epoch': 200, 'batch_size': 1024, 'dataloader_workers': 32, 'type': 'normal', 'note': '', 'lr': 0.05}, 'adversarial_training_params': {'eps': 0.03, 'name': 'PGD'}}


Testing: 0it [00:00, ?it/s]

2023-11-25 07:03:42,934 [INFO]: test_clean_accuracy: 0.8744000196456909
2023-11-25 07:03:42,937 [INFO]: test_adv-PGD-eps0.01_accuracy: 0.09740000218153
2023-11-25 07:03:42,938 [INFO]: test_adv-PGD-eps0.03_accuracy: 0.0
2023-11-25 07:03:42,939 [INFO]: test_adv-PGD-eps0.1_accuracy: 0.0
2023-11-25 07:03:42,940 [INFO]: test_adv-PGD-eps0.2_accuracy: 0.0
2023-11-25 07:03:42,941 [INFO]: test_adv-PGD-eps0.3_accuracy: 0.0
2023-11-25 07:03:42,942 [INFO]: test_adv-PGD-eps0.5_accuracy: 0.0
2023-11-25 07:03:42,945 [INFO]: test_adv-FGSM-eps0.01_accuracy: 0.2296999990940094
2023-11-25 07:03:42,945 [INFO]: test_adv-FGSM-eps0.03_accuracy: 0.03060000017285347
2023-11-25 07:03:42,946 [INFO]: test_adv-FGSM-eps0.1_accuracy: 0.049300000071525574
2023-11-25 07:03:42,947 [INFO]: test_adv-FGSM-eps0.2_accuracy: 0.10570000112056732
2023-11-25 07:03:42,949 [INFO]: test_adv-FGSM-eps0.3_accuracy: 0.10339999943971634
2023-11-25 07:03:42,951 [INFO]: test_adv-FGSM-eps0.5_accuracy: 0.10000000149011612
2023-11-25 07:03:

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         Test metric                   DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
test_adv-FGSM-eps0.01_accuracy      0.2296999990940094
test_adv-FGSM-eps0.03_accuracy     0.03060000017285347
test_adv-FGSM-eps0.1_accuracy      0.049300000071525574
test_adv-FGSM-eps0.2_accuracy      0.10570000112056732
test_adv-FGSM-eps0.3_accuracy      0.10339999943971634
test_adv-FGSM-eps0.5_accuracy      0.10000000149011612
test_adv-PGD-eps0.01_accuracy        0.09740000218153
test_adv-PGD-eps0.03_accuracy              0.0
 test_adv-PGD-eps0.1_accuracy              0.0
 test_adv-PGD-eps0.2_accuracy              0.0
 test_adv-PGD-eps0.3_accuracy              0.0
 test_adv-PGD-eps0.5_accuracy              0.0
     test_clean_accuracy            0.8744000196456909
───────────────────────────────

In [13]:
#check evaluation pointwise
print('==========================')
model.eval()
test_dataloader = model.test_dataloader()
acc = torchmetrics.Accuracy(task='multiclass', num_classes=10)
lst_acc = []
for batch in test_dataloader:
    x,y = batch
    with torch.inference_mode():
        logits = model(x)
    predicted = torch.argmax(logits, dim=1)
    lst_acc.append(acc(predicted, y))

test_acc = torch.Tensor(lst_acc).mean()
print(f'Test accuracy {test_acc:.4f} ({test_acc*100:.2f}%)')

Test accuracy 0.8744 (87.44%)
